In [2]:
import os
os.getcwd()
os.chdir(r'/home/featurize/data/Bert/')

In [2]:
from datasets import Dataset
import pandas as pd

#读取数据并shuffle
train_df = pd.read_csv('../data/train_set.csv', sep='\t')
test_df = pd.read_csv('../data/test_a.csv', sep='\t')

#将训练数据中三个token换成标点
# train_df['texts']=train_df['text'].map(lambda x:replacepunc(x))

/environment/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_datasets = Dataset.from_pandas(train_df)
test_datasets = Dataset.from_pandas(test_df)

In [4]:
#准备将text文本首尾截断，各取255tokens
# cnt=0
def slipt2(x):
    new_texts = []
    texts = x['text']
    for text in texts:
        ls = text.split(' ')
        le = len(ls)
        if le < 511:
            new_texts.append(text)
        else:
            new_texts.append(' '.join(ls[:255] + ls[-255:]))
    # 返回一个字典，键为列名 'text'，值为处理后的文本列表
    return {'text': new_texts}
    
train_datasets = train_datasets.map(lambda x: slipt2(x),batched=True, num_proc=8, batch_size=5000)
test_datasets = test_datasets.map(lambda x: slipt2(x),batched=True, num_proc=8, batch_size=5000)

Map (num_proc=8): 100%|██████████| 50000/50000 [00:12<00:00, 3880.94 examples/s]


In [3]:
from transformers import BertTokenizer
vocab_file_path = 'vocab.txt'
# 以只读模式打开文件
with open(vocab_file_path, 'r', encoding='utf-8') as file:
    # 读取文件的所有行
    lines = file.readlines()
    # 计算行数
    vocab_size = len(lines)

# 打印 vocab_size 的值
print(f"vocab_size: {vocab_size}")

tokenizer = BertTokenizer(vocab_file=vocab_file_path)

# tokenized_dataset = train_datasets.map(lambda examples: tokenizer(examples['text']), batched=True, num_proc=16, batch_size=5000).remove_columns("text")
# tokenized_test_dataset = test_datasets.map(lambda examples: tokenizer(examples['text']), batched=True, num_proc=16, batch_size=5000).remove_columns("text")

/environment/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


vocab_size: 6982


In [6]:
train_size = train_datasets.shape[0]//10*8
test_size = train_datasets.shape[0]//10*2
datasets = tokenized_dataset.train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 160000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40000
    })
})

In [4]:
#加载和保存拼接后的文本，掉线的时候这么做
# datasets.save_to_disk('./fineTune_data/train')
# tokenized_test_dataset.save_to_disk('./fineTune_data/test')
import pandas as pd
from datasets import load_from_disk
datasets=load_from_disk('./fineTune_data/train')
# datasets=load_from_disk('./fineTune_data/train')

In [5]:

from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained("./preTrain_log/checkpoint-69420",num_labels=14)

#使用GPU训练
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


2025-01-31 06:43:43.330616: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-31 06:43:43.360437: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-31 06:43:43.360465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-31 06:43:43.361318: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-31 06:43:43.366922: I tensorflow/core/platform/cpu_feature_guar

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(6982, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12

In [7]:

import random
import numpy as np
import torch

# 固定所有随机种子
seed = 1222  # 可替换为任意整数
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # 确保CUDA卷积操作确定性
    torch.backends.cudnn.benchmark = False     # 关闭自动寻找最优卷积算法（可能引入随机性）

from transformers import TrainingArguments

# 在 TrainingArguments 中设置种子
training_args = TrainingArguments(
    output_dir="fineTune_log",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=36,
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    seed=seed  # 关键：显式设置训练用种子
)

# ... 后续代码保持不变 ...

from transformers import Trainer
# 从 evaluate 库加载指标
import evaluate
metric = evaluate.load("f1")

# 定义 compute_metrics 函数时显式指定宏平均，并打印分类报告
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    # 计算宏平均 F1 分数
    f1_score = metric.compute(predictions=predictions, references=labels, average="macro")
    # 计算分类报告
    report = classification_report(labels, predictions)
    print("分类报告：")
    print(report)
    return f1_score

trainer = Trainer(
    model=model,
    args=training_args, 
    train_dataset=datasets["train"],
    eval_dataset=datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


/environment/miniconda3/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_26669/955321044.py:50: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1
1,0.227800,0.201210,0.922405
2,0.190800,0.175672,0.933631
3,0.169700,0.166380,0.937985
4,0.140800,0.161812,0.940520
5,0.124000,0.161351,0.940556
6,0.116200,0.157439,0.943630
7,0.109100,0.160037,0.943946
8,0.092100,0.156880,0.944824
9,0.081300,0.163417,0.942785
10,0.077100,0.173383,0.940969


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f340645c910>>
Traceback (most recent call last):
  File "/environment/miniconda3/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [9]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_from_disk
import numpy as np

# 步骤 1: 加载检查点模型
checkpoint_path = './fineTune_log/checkpoint-5000'  # 根据实际路径修改
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

# 步骤 2: 初始化 Trainer
# 定义训练参数，这里主要用于预测或评估
training_args = TrainingArguments(
    output_dir='./',
    per_device_eval_batch_size=256,  # 可根据实际情况调整
    # 这里不需要训练相关的参数，因为只是使用模型进行预测或评估
    do_train=False
)

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer
)

# 步骤 3: 进行后续操作

# 示例 1: 预测并保存结果
# 加载测试数据集
tokenized_test_ds = load_from_disk('./fineTune_data/test')

# 进行预测
predictions, metrics, loss = trainer.predict(tokenized_test_ds, metric_key_prefix="test")

# 获取预测标签
pred = np.argmax(predictions, axis=1)

# 保存预测结果到 CSV 文件
pd.DataFrame({'label': pred}).to_csv('./submission_Bert.csv', index=None)

/tmp/ipykernel_26669/4246783075.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
